## Recurrent Neural Networks in PyTorch

The Recurrent Neural Network (RNN) Architecture, which can actually be seen as a strict generalization of standard MLPs is, alongside Convolutional Neural Networks, one of the classic Architectures of Neural Networks. 

In some sense, RNN's introduce a time component into the Neural Network by not needed to compute data in a single forward pass. For a basic Introduction see https://en.wikipedia.org/wiki/Recurrent_neural_network.

They also have many interesting Theoretical properties, like being able to model any Computable function :O  which we will not focus on. Because it is so widely used, there already exists a PyTorch implementation. 

In [1]:
import torch
import torch.nn as nn

In [ ]:
class basicRNN(nn.Module):
    def __init__(self, hidden_dim, input_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.act_hidden = nn.Tanh()
                
        self.in_hidden = nn.Parameter(torch.randn(hidden_dim,input_dim))
        self.hidden_hidden = nn.Parameter(torch.randn(hidden_dim,hidden_dim))
        self.hidden_out = nn.Parameter(torch.randn(output_dim,hidden_dim))
    
    def forward(self, x, h_t = None): 
        if h_t is None: 
            h_t = torch.zeros(self.hidden_dim) 
        
        out = []
        for t in range(x.size(0)):
            x_t = x[t] # value at current time-step
            h_next = self.act_hidden(self.in_hidden @ x_t + self.hidden_hidden @ h_t)
            y = self.hidden_out @ h_next
            out.append(y)
        out = torch.stack(out, dim = 0)
        return out, h_next               